# Phase 4 Project


**Authors**: [Dominic Muli](mailto:dominic.muli@student.moringaschool.com), [Evangeline Ngunjiri](mailto:evangeline.ngunjiri@student.moringaschool.com), and [Ivy Ndunge](mailto:ivy.ndunge@student.moringaschool.com)

## 1.0) Project Overview

## 1.1) Business Problem

## 1.2) Objectives

## 1.3) Metric of Success

## 1.4) Understanding Data


## 1.5) Data Relevance and Validation

## 2.0) Reading and Understanding the Data

In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt


In [ ]:
# reading the data


## 2.1) Data Cleaning

## 2.2) EDA

## 2.3) Building Model

## 2.4) Conclusion

## 2.5) Recommendation